# Load ERA5-Land air temperature and precipitation data, save as .csv at each study site

In [1]:
import ee
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tqdm.auto import tqdm

In [2]:
# path to snow_cover_mapping_application/
# base_path = 'drive/MyDrive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/'

# path to study-sites/
study_sites_path = '/Users/raineyaberle/Google Drive/My Drive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'


In [3]:
# If using Google Colab, mount Google Drive so you can access your Drive folders
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Authenticate and intialize Google Earth Engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# Uncomment and use this to locate the folder above, if needed
# os.listdir('drive/MyDrive/Research/PhD/snow_cover_mapping/snow_cover_mapping_application/study-sites/')

In [5]:
# Grab list of study site names in folder
os.chdir(study_sites_path)
site_names = sorted([x[0:-1] for x in glob.glob('*/', recursive = True)])
site_names

['Blue',
 'Boulder',
 'Carbon',
 'Coleman',
 'Easton',
 'Emmons',
 'Gulkana',
 'Hidden',
 'Hoh',
 'LemonCreek',
 'RGI60-01.00037',
 'RGI60-01.00038',
 'RGI60-01.00046',
 'RGI60-01.00312',
 'RGI60-01.00566',
 'RGI60-01.00576',
 'RGI60-01.01151',
 'RGI60-01.01390',
 'RGI60-01.01524',
 'RGI60-01.01733',
 'RGI60-01.03861',
 'RGI60-01.04375',
 'RGI60-01.06268',
 'RGI60-01.06722',
 'RGI60-01.08155',
 'RGI60-01.08174',
 'RGI60-01.08246',
 'RGI60-01.08248',
 'RGI60-01.08262',
 'RGI60-01.08288',
 'RGI60-01.08296',
 'RGI60-01.08302',
 'RGI60-01.08336',
 'RGI60-01.08353',
 'RGI60-01.08389',
 'RGI60-01.08395',
 'RGI60-01.08403',
 'RGI60-01.08412',
 'RGI60-01.08427',
 'RGI60-01.09148',
 'RGI60-01.09216',
 'RGI60-01.09411',
 'RGI60-01.09639',
 'RGI60-01.10196',
 'RGI60-01.10555',
 'RGI60-01.10689',
 'RGI60-01.10778',
 'RGI60-01.10851',
 'RGI60-01.10857',
 'RGI60-01.11616',
 'RGI60-01.11654',
 'RGI60-01.11788',
 'RGI60-01.12370',
 'RGI60-01.12425',
 'RGI60-01.12635',
 'RGI60-01.13696',
 'RGI60-01.143

In [ ]:
# Define date and month ranges for ERA5 querying
date_start = '2010-05-01'
date_end = '2023-11-01'

# Iterate over study sites
for site_name in ['Sperry']:

    print(site_name)

    # Load AOI (glacier outline)
    AOI_fn = glob.glob(site_name + '/AOIs/*_outline.shp')[0]
    AOI = gpd.read_file(AOI_fn)
    # reproject to WGS84
    AOI_WGS = AOI.to_crs('EPSG:4326')

    # Reformat AOI for GEE querying
    region = ee.Geometry.Polygon([[[AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]],
                                  [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.miny[0]],
                                  [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.maxy[0]],
                                  [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.maxy[0]],
                                  [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]]
                                  ]])

    # Define the dataset
    era5_land = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
                                  .filter(ee.Filter.date(date_start, date_end))
                                  .filter(ee.Filter.calendarRange(5, 11, 'month'))
                                  .filterBounds(region)
                                  )

    # Convert timestamps to a list of 'YYYY-MM-dd'
    timestamps = era5_land.aggregate_array('system:time_start')
    timestamps_list = ee.List(timestamps).map(lambda timestamp: ee.Date(timestamp).format("YYYY-MM-dd")).getInfo()

    # Initialize an empty list to hold temperature and precipitation values
    temperature_list = []
    precipitation_list = []

    # Iterate over each timestamp and extract temperature and precip for that day
    for timestamp in tqdm(timestamps_list):
        # Convert the timestamp to an EE Date object
        ee_timestamp = ee.Date(timestamp)

        # Filter the image collection for the specific date
        image_of_the_day = era5_land.filter(ee.Filter.date(ee_timestamp, ee_timestamp.advance(1, 'day'))).first()

        # Calculate the mean temperature over the region for the specific date
        mean_temperature = image_of_the_day.select('temperature_2m').subtract(273.15).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=1000  # Adjust the scale as needed
        ).get('temperature_2m')

        # Calculate the mean precipitation over the region for the specific date
        mean_total_precpitation = image_of_the_day.select('total_precipitation_sum').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=1000  # Adjust the scale as needed
            ).get('total_precipitation_sum')

        # Append the values to the list
        temperature_list.append(mean_temperature.getInfo())
        precipitation_list.append(mean_total_precpitation.getInfo())

    # Create a Pandas DataFrame
    df = pd.DataFrame({'Date': timestamps_list,
                       'Mean_Temperature_Celsius': temperature_list,
                       'Mean_Total_Precipitation_Meters': precipitation_list})
    df['Date'] = pd.to_datetime(df['Date'])

    # Calculate Positive Degree Days (PDDs)
    df['Positive_Degree_Days'] = df['Mean_Temperature_Celsius'].apply(lambda x: max(0, x))
    # Calculate cumulative sum and reset at the start of each calendar year
    df['Cumulative_Positive_Degree_Days'] = df.groupby(df['Date'].dt.year, group_keys=True)['Positive_Degree_Days'].cumsum()
    # Reset cumulative sum to zero at the start of each year
    df['Cumulative_Positive_Degree_Days'] = df.groupby(df['Date'].dt.year, group_keys=False)['Cumulative_Positive_Degree_Days'].apply(lambda x: x - x.iloc[0])

    # Reorder columns
    df = df[['Date', 'Mean_Temperature_Celsius',
             'Mean_Total_Precipitation_Meters',
             'Positive_Degree_Days', 'Cumulative_Positive_Degree_Days']]

    # Create directory for outputs if it does not exist
    out_path = site_name + '/' + 'ERA/'
    if not os.path.exists(out_path):
        os.mkdir(out_path)
        print('Made directory for outputs: '+out_path)

    # Save DataFrame to CSV
    output_fn = out_path + site_name + '_ERA5_PDDs_' + date_start + '_' + date_end + '.csv'
    df.to_csv(output_fn, index=False)
    print('ERA mean air temperature and PDDs saved to file: ' + output_fn)

    # Plot mean air temperatures and PDDs
    fig, ax = plt.subplots(2, 1, figsize=(12,10))
    ax[0].plot(df.Date.values.astype('datetime64[ns]'), df.Mean_Temperature_Celsius.values, '.k')
    ax[0].set_ylabel('Mean daily 2m air temperature [C]')
    ax[0].grid()
    ax2 = ax[0].twinx()
    ax2.plot(df.Date.values.astype('datetime64[ns]'), df.Cumulative_Positive_Degree_Days, '.m')
    ax2.set_ylabel('$\Sigma$ Positive Degree Days')
    ax[1].plot(df.Date.values.astype('datetime64[ns]'), df.Mean_Total_Precipitation_Meters, '.b')
    ax[1].grid()
    ax[1].set_xlabel('Date')
    ax[1].set_ylabel('Daily Mean Total Precipitation [m]')
    plt.show()

    # save figure
    fig_fn = out_path + site_name + '_ERA_PDDs_precip.png'
    fig.savefig(fig_fn, dpi=300)
    print('figure saved to file: ' + fig_fn)
    print(' ')
